# Import Dependencies

In [8]:
import os
import kaggle

     |████████████████████████████████| 82 kB 991 kB/s eta 0:00:011
     |████████████████████████████████| 167 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 78 kB 1.6 MB/s eta 0:00:01
  Created wheel for kaggle: filename=kaggle-1.6.17-py3-none-any.whl size=105790 sha256=ec28a7f82c9739311a9fbd40f3c8031ce9725c595175a3f367fa4f2bcf958f35
  Stored in directory: /home/sihala/.cache/pip/wheels/a5/6f/7b/837915771e94e181fa3052822926444e34f725ca38e70be77e
Successfully built kaggle


# Load Kaggle Dataset

In [88]:
kaggle_token_path = os.path.dirname(os.getcwd())+"/token"
data_path = os.path.dirname(os.getcwd())+"/data"
os.environ["KAGGLE_CONFIG_DIR"] = kaggle_token_path
os.chmod(kaggle_token_path+"/kaggle.json", 0o600)



current_dir = os.getcwd()
os.chdir(data_path)
!kaggle datasets download -d fabianavinci/guitar-chords-v3 --unzip
os.chdir(current_dir)

Dataset URL: https://www.kaggle.com/datasets/fabianavinci/guitar-chords-v3
License(s): unknown
 33%|█████████████▎                          | 242M/729M [00:04<00:08, 61.1MB/s]^C
 33%|█████████████▎                          | 243M/729M [00:04<00:08, 58.1MB/s]
User cancelled operation


# Load IDMT-SMT Guitar Dataset

In [91]:
import requests
from tqdm import tqdm

os.chdir(data_path)
url = ("https://zenodo.org/records/7544213/files/IDMT-SMT-CHORDS.zip?download=1")
download_zip = "IDMT-SMT-CHORDS.zip"

if download_zip not in os.listdir(data_path):
    response = requests.get(url, stream=True)
    
    # Sizes in bytes.
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024
    
    with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
        with open(download_zip, "wb") as file:
            for data in response.iter_content(block_size):
                progress_bar.update(len(data))
                file.write(data)
    
    if total_size != 0 and progress_bar.n != total_size:
        raise RuntimeError("Could not download file")
        


100%|██████████████████████████████████████| 1.31G/1.31G [13:14<00:00, 1.64MB/s]


In [92]:
import zipfile

file_path = '../data/{}'.format(download_zip)
extract_path = '../data/{}'.format("IDMT")


with zipfile.ZipFile(file_path, 'r') as zip_ref:
    files = zip_ref.namelist()

    for file in tqdm(files, desc="Extracting files", unit = "file"):
        zip_ref.extract(file, extract_path)

Extracting files: 100%|███████████████████████| 24/24 [00:08<00:00,  2.86file/s]


# Get only guitar audio and remove other unnecessary files

In [93]:
import shutil

guitar_data_path = extract_path+"/IDMT-SMT-CHORDS/guitar"

if os.path.exists(guitar_data_path):
    shutil.move(guitar_data_path, extract_path)

if os.path.exists(extract_path+"/IDMT-SMT-CHORDS"):
    shutil.rmtree(extract_path+"/IDMT-SMT-CHORDS")

# Re-sample IDMT audio to match the sample rate of the Kaggle dataset

In [94]:
#The IDMT audio has a sample rate of 44.1 kHz, while the Kaggle dataset is only 16 kHz. 
#We can re-sample the higher rate files and bring them down to match the other dataset.
import librosa
import soundfile as sf

guitar_path = extract_path + "/guitar"


# Load the audio file at its original sample rate
original_sample_rate = 44100
target_sample_rate = 16000


#Loop through and resampe .wav files
for filename in os.listdir(guitar_path):
    file_path = os.path.join(guitar_path, filename)
    if os.path.isfile(file_path) and '.wav' in file_path:
        audio_data, sr = librosa.load(file_path, sr=original_sample_rate)
        audio_resampled = librosa.resample(audio_data, orig_sr=sr, target_sr=target_sample_rate)
        sf.write(guitar_path+'/resampled_{}'.format(filename), audio_resampled, target_sample_rate)
        os.remove(file_path)

In [114]:
#Rename files to remove unnecessary details
for filename in os.listdir(guitar_path):
    chord_data_path = os.path.join(guitar_path, filename)
    if os.path.isfile(chord_data_path) and '.wav' in chord_data_path:
        new_name = filename.replace("_ableton_live_guitar", "")
        new_name = filename.replace("_garageband_guitar", "")
        new_name = filename.replace("resampled_", "")
        os.rename(chord_data_path, os.path.join(guitar_path, new_name))

# Associate audio timestamps with chord labels

In [113]:
timestamp_mappings = {}
with open(guitar_path+"/guitar_annotation.lab", 'r') as file:
    for line in file:
        start, end, chord = line.split()
        timestamp_mappings[end] = chord

# Split audio into individual chord clips

In [129]:
from scipy.io import wavfile

def split_audio_file(input_file_name: str, input_dir_path: str):
    start_split = 0
    split_at_timestamp = 2
    rate, data = wavfile.read(os.path.join(input_dir_path, input_file_name)) 
    duration = len(data)//rate
    num_clips = duration // 2
    
    
    for i in range(num_clips):    
    
        # get the frame to split at
        split_at_frame = rate * split_at_timestamp
        start_at_frame = rate *start_split
    
        # split
        left_data = data[start_at_frame:split_at_frame-1]
    
        # save the result
        output_dir = os.path.join(input_dir_path, "split")
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
                             
        original_file_name = input_file_name.replace('.wav','')
        output_name = "{}_{}.wav".format(original_file_name, split_at_timestamp)
        
        wavfile.write(os.path.join(output_dir,output_name), rate, left_data)
        start_split+=2
        split_at_timestamp+=2

In [130]:
for filename in os.listdir(guitar_path):
    chord_data_path = os.path.join(guitar_path, filename)
    if os.path.isfile(chord_data_path) and '.wav' in chord_data_path:
        split_audio_file(filename, guitar_path)